### TP3


1. Pretende-se construir uma implementação simplificada do algoritmo “model checking” orientado aos interpolantes seguindo a estrutura apresentada nos apontamentos onde no passo $(n,m)\,$na impossibilidade de encontrar um interpolante invariante se dá ao utilizador a possibilidade de incrementar um dos índices $n$ e $m$ à sua escolha.
    Pretende-se aplicar este algoritmo ao problema da da multiplicação de inteiros positivos em `BitVec`  (apresentado no TP2).

In [1]:
from z3 import *

<img src="automato.png" alt="autómato" style="width: 600px;"/>

#### Função declare(t, N):

   A seguinte função cria a $t$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.

In [2]:
def declare(i, n):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = BitVec('x'+str(i), n)
    state['y'] = BitVec('y'+str(i), n)
    state['z'] = BitVec('z'+str(i), n)
    
    return state

#### Função init(state, prob, N):

    state - Dicionário de variáveis de estado
    a - valor associado ao 1º número a multiplicar
    b - valor associado ao 2º número a multiplicar

   A função `init` tem como objetivo devolver um predicado do Solver que testa se é um possível estado inicial do programa, através do `state`, um dicionário de variáveis.

In [35]:
def init(state, a, b):
    return And(state['pc']== 0, state['x'] == a, state['y'] == b, state['z'] == 0)

#### Função trans(curr, prox, N):

    curr - Estado das variáveis no momento atual
    prox - Estado das variáveis no momento da próxima iteração
    n - Número de bits 
    
   A função `trans` tem como objetivo devolver um predicado do Solver, através dos três estados disponíveis, que teste se é possível transitar entre os estados possíveis.

In [38]:
def error1(state):
    return Or(Equals(state['pc'] ,4), Equals(state['pc'] ,6))

def trans(curr, prox, n):
    same_values = And(
        prox['x'] == curr['x'],
        prox['y'] == curr['y'],
        prox['z'] == curr['z']    
    )
    
    t0 = And(
        curr['pc'] == 0,
        prox['pc'] == 1,
        same_values
    )
    
    # y = 0
    t1 = And(
        curr['y'] == 0,
        curr['pc'] == 1,
        prox['pc'] == 5,
        same_values
    )
    
    # y != 0 ^ odd(y)
    t2 = And(
        curr['y'] != 0,
        URem(curr['y'], 2) == 1,
        curr['pc'] == 1,
        prox['pc'] == 2,
        same_values
    )
    
    
    # y != 0 ^ even(y)
    t3 = And(
        curr['y'] != 0,
        URem(curr['y'], 2) == 0,
        curr['pc'] == 1,
        prox['pc'] == 3,
        same_values
    )

    

    # transição em que o solver decide se vai para o estado de overflow ou se continua
    magia_left = And(
        prox['x'] == curr['x'] << BitVecVal(1, n),
        prox['y'] == curr['y'] >> BitVecVal(1, n),
        prox['z'] == curr['z'],
        
        curr['pc'] == 3,
        
        Or(
            And(ULT(curr['x'], prox['x']), prox['pc'] == 1),  # curr['x'] < prox['x']  - não há overflow
            And(UGE(curr['x'], prox['x'] ), prox['pc'] == 4),  # curr['x'] >= prox['x'] - há overflow
        )
    )
    
    # transição em que o solver decide se vai para o estado de overflow ou se continua
    magia_right = And(
        prox['x'] == curr['x'],
        prox['y'] == curr['y'] - BitVecVal(1,n),
        prox['z'] == curr['z'] + curr['x'],
        
        curr['pc'] == 2,
        
        Or(
            And(UGT(prox['z'], curr['z']), prox['pc'] == 1),  # curr['x'] < prox['x']  - não há overflow
            And(ULT(prox['z'], curr['z']), prox['pc'] == 6),  # curr['x'] >= prox['x'] - há overflow
        )
    )
    
    
    # caso de paragem no overflow e no estado final
    stop_case = And(
        prox['pc'] == curr['pc'],
        same_values,
        
        Or(
            And(curr['pc'] == 4, prox['pc'] == 4),
            And(curr['pc'] == 5, prox['pc'] == 5),
            And(curr['pc'] == 6, prox['pc'] == 6)
        )
    )
    
    
    return Or(t0, t1, t2, t3, stop_case, magia_left, magia_right)

#### Função gera_traco(declare, init, trans, k, n, a, b)

    declare - Variáveis de estado
    init - Condições para o estado inicial
    trans - Função transição
    k - Valor do traço
    n - Número de bits
    a - Valor a ser multiplicado
    b - Valor a ser multiplicado

A função `gera_traco` tem como objetivo imprimir o valor das variáveis à medida que vão percorrendo os estados, através das variáveis do estado, de um predicado que testa se um estado é inicial, um número positivo para gerar um possível traço de execução do programa de tamanho `k` , com `N` bits, multiplicando `a` por `b`.

In [43]:
def gera_traco(declare,init,trans,k, n, a, b):

    s = Solver()
    
    
    X = [declare(i, n) for i in range(k)]

    # adicionar o estado inicial
    s.add(init(trace[0],a,b))
    
    # adicionar as transições
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i+1], n))
    
    
    check = s.check()
    if check == sat:
        m = s.model()
        #print(m)
        #print(trace[i])
        for i in range(k):
            print("Passo ", i)
            for v in trace[i]:
                
                print(v, "=", m[trace[i][v]])
            print("----------------")
    else:
        print(check)
                
gera_traco(declare,init,trans,20, 3, 7, 2)

Passo  0
pc = 0
x = 7
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 7
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 7
y = 2
z = 0
----------------
Passo  3
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  4
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  5
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  6
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  7
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  8
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  9
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  10
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  11
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  12
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  13
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  14
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  15
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  16
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  17
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  18
pc = 4
x = 6
y = 1
z = 0
----------------
Passo  19
pc = 4
x = 6